In [133]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Statistics

This notebook contains plots and other code to explore guesses of the players of the game.

### Table of Content:

* ['Guesses' stats](#guess-stats)
* [Ground truth related stats](#ground-truth-stats)


In [134]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
%run ./prepare_data.ipynb






## <a class="anchor" id="guess-stats">Guesses stats</a>

### Most popular and least votes

In [135]:
df_guess_simple = df_guess
#df_guess_simple['Current URL'] = df_guess['Current URL'].str.replace('^[^_]*=', '')\
#df_guess_simple['Current URL'] = df_guess['Current URL'].str.replace('https://youtu.be/', '')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_guess_simple)

   Current URL     Guess  votes
0        URL10   Player4      4
1        URL10   Player6      3
2        URL10   Player3      2
3        URL11   Player1      2
4        URL11   Player9      2
5         URL6   Player6      2
6        URL10   Player1      1
7        URL10   Player2      1
8        URL11  Player10      1
9        URL11  Player11      1
10       URL11   Player2      1
11       URL11   Player3      1
12       URL11   Player5      1
13       URL11   Player7      1
14       URL11   Player8      1
15        URL6   Player3      1


## <a class="anchor" id="ground-truth-stats">Ground truth related stats </a>

First, we compute the intersection between players/url column of ground truth df and guesses/url column of guess df, for all URLs.
Note that one guess is inherently incorrect (by the rules of the game).

In [136]:
#FIXME: this is not pretty

# Correct guesses
correct_guess_df = pd.merge(df_raw,df2_raw, how='inner', left_on=['Current URL', 'Guess'], right_on=['URL','Player'])
del correct_guess_df['Player_y'], correct_guess_df['URL']
correct_guess_df.rename(columns = {'Player_x':'Player'}, inplace = True)
correct_guess_df.rename(columns = {'Current URL':'URL'}, inplace = True)
correct_guess_df

,URL,Player,Guess
0,URL11,Player8,Player11
1,URL6,Player9,Player6
2,URL6,Player1,Player6


In [137]:
# Correct guesses per player
#TODO: display this better

corr_guess_stats = correct_guess_df.groupby(by=['Player'])['Guess'].count()
corr_guess_stats = pd.merge(df2_raw, corr_guess_stats, on='Player',how='left').fillna(0)
del corr_guess_stats['URL']
print(corr_guess_stats)

      Player  Guess
0    Player1    1.0
1    Player2    0.0
2    Player3    0.0
3    Player4    0.0
4    Player5    0.0
5    Player6    0.0
6    Player7    0.0
7    Player8    1.0
8    Player9    1.0
9   Player10    0.0
10  Player11    0.0


In [138]:
# Best and worst guessed players
#TODO: display this better

corr_player_stats = correct_guess_df.groupby(by=['URL'])['Guess'].count()
corr_player_stats = pd.merge(df2_raw, corr_player_stats, on='URL',how='left').fillna(0)
del corr_player_stats['URL']
print(corr_player_stats)


      Player  Guess
0    Player1    0.0
1    Player2    0.0
2    Player3    0.0
3    Player4    0.0
4    Player5    0.0
5    Player6    2.0
6    Player7    0.0
7    Player8    0.0
8    Player9    0.0
9   Player10    0.0
10  Player11    1.0
